# Simulation of quantum systems

In this notebook, we introduce the basic concepts of quantum simulation. 

# Installation

In [ ]:
%pip install simuq

We start with programming the Ising model on a 3 qubit chain.

Prepare the python environment


In [ ]:
from simuq.qsystem import QSystem, qubit


Here QSystem is the class for quantum systems, and qubit is the class for qubit sites.

## Define the evolution

First we create a quantum system and a list of qubit sites.

In [ ]:
qs = QSystem()
q = [qubit(qs) for i in range(n)]

Our target is a short evolution governed by an Ising Hamiltonian $H=X_1X_2+X_2X_3+Z_1+Z_2+Z_3$ over the list of qubits, which can be programmed by


In [ ]:
h = q[0].X * q[1].X + q[1].X * q[2].X + q[0].Z + q[1].Z + q[2].Z




We add a T=1 time evolution under H to the quantum system.


In [ ]:
T = 1
qs.add_evolution(h, T)

Then `qs` contains the evolution of $H$ for time $T$. To add next segment of evolution, we only need to call the method `add_evolution` again.

# Create a QuTiP provider

A provider is a user interface for convenient manipulations of functionalities of SimuQ. We use QuTiP provider as a basic example on how to use providers to deploy quantum simulation problems on devices and obtain results.

We can create a QuTiP provider via the following code


In [ ]:
from simuq.providers import QuTiPProvider
qpp = QuTiPProvider()

Compilation in provider

To simulate a quantum system qs programmed in HML via SimuQ, we need three major steps of a provider: compile, run, results.

We call the compile function of the provider to process the system into a runnable executable. For QuTiP provider, we can execute

In [ ]:
qpp.compile(qs)

QuTiP provider processes the quantum system qs and translate it into a Hamiltonian in QuTiP. When succeeds, a message Compiled. will show up.

For other providers, compile command may specify the backend device, AAIS, and compiler specifications.

When compilation succeeds, the job will be recorded in the provider.

Run and obtain results from providers

Running a job will send the compilation results to backend devices to execute. For QuTiP provider, we execute


In [ ]:

qpp.run()

When successfully executed, the command will show a message Submitted, meaning that the executables are sent to devices to run.

For other providers, you can choose whether to run on classical simulators, and how many shots to run on the real device or simulators. Note that these classical simulators are mostly from hardware providers and executed on the cloud.

To retrieve the results, we can execute

In [ ]:


qpp.results()


If the task is still in the queue on the device, an exception will be raised. When the retrieval succeeds, a dictionary is returned, which contains the frequencies of obtaining a measurement array (encoded as a 0/1 string). A bit in the string corresponds to a site of the quantum system. We can call qpp.print_sites() to show the order of the sites in the measurement output.

The resulting dictionary of the 3-site Ising model is

# Declare IonQ providers

Accesses to IonQ Quantum Cloud require an API key from IonQ.

Assuming that you have a string API_key storing it, we can declare an IonQ provider in SimuQ by

In [ ]:


from simuq.providers import IonQProvider
iqp = IonQProvider(API_key)

Compile, run, and retrieve results

To compile and run the 3-site Ising system qs in programming an Ising model on IonQ devices, we need to specify a device and an AAIS. By default, the device is Aria-1 and the AAIS in compilation is Heisenberg-AAIS. One can also pass detailed parameters in the compilation like error tolerance bound and number of Trotterization here. For simplicity, we execute with the default compiler configuration by

In [ ]:

iqp.compile(qs)

We then run the experiment on simulator, which executes


In [ ]:
iqp.run(on_simulator = True)


This will submit a job to IonQ Quantum Cloud using the compiled results, and print the job id and status. The simulator will account for the noise model of Aria-1. To retrieve the results, we wait for a short period of time (normally several seconds) for the cloud server to queue and process the simulation, and then run


In [ ]:
iqp.results()


This returns a dictionary, whose results may depend on calibration data changing by date. In our experiments, it returns


To run on real devices, we can execute



In [ ]:
iqp.run(shots = 4096)


Here shots represent the number of repetition of the experiment. After queuing and executing, you may retrieve the results by calling iqp.results().